# Include the CMB probes in the Euclid likelihood

We can then compare the log posterior in the 3x2pt and the 7x2pt cases


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import os, sys

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

import cloe
from cloe.cobaya_interface import EuclidLikelihood
from cloe.auxiliary.logger import log_warning
from cloe.photometric_survey.photo import Photo
from cloe.cmbx_p.cmbx import CMBX

from cobaya.model import get_model
from cobaya.run import run
import time
import copy

In [ ]:
import matplotlib as mpl

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['font.size'] = 20
mpl.rcParams['figure.figsize'] = 8.5, 5.5

mpl.rcParams['errorbar.capsize'] = 4

mpl.rc('legend', fontsize=15)

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running
# This dictionary below ('info') can be modified up to some point by the user to
# adapt it to the user's needs.
# The options that can be modified by the user are pointed with the acronym (UC).

nlflag=0
barflag=0

# Set this to zero for linear Cl_kk from CAMB
# But the Cl_kk will be inacurate for L>1000
# We use it only for comparison with CLOE
nlflag_camb=0

info = {
    #'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
'params': {
        # (UC): each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
        # with the key 'prior', 'latex'...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the parameter ns
        # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67.0, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu':{"prior": {"min": 0., "max": 1.}}, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
#         'ns': {'prior':{'min':0.8, 'max':1.2}}, # primordial power spectrum tilt (sampled with an uniform prior)
#         'As':{"prior": {"min": 2.10e-9, "max": 2.15e-9}},
        'ns': 0.96, # primordial power spectrum tilt 
        'w': -1.0, #Dark energy fluid model
        'wa': 0.0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    #'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # (UC): you can choose between CAMB or CLASS
    # In this DEMO, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    # ATTENTION: If you have CAMB/CLASS already installed and 
    # you are not using the likelihood conda environment 
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    # NOTE: for values of the non-linear flag larger than 0, a new key is added in info['theory']['camb']['extra_args'],
    # i.e. 'halofit_version', which contains the requested version of halofit, as described above
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf', 
                             'lens_potential_accuracy': nlflag_camb}}},

    
    #'sampler': Cobaya's protected key of the input dictionary.
    # (UC): you can choose the sampler you want to use.
    # Check Cobaya's documentation to see the list of available samplers
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    # Note: if you want to run a simple MCMC sampling choose 'mcmc'
    'sampler': {'evaluate': None},  
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files? 
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/my_euclid_experiment',
    #'debug': Cobaya's protected key of the input dictionary.
    # (UC): how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': True,
    #'force': Cobaya's protected key of the input dictionary.
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True,
    }

In [ ]:
#'likelihood': Cobaya's protected key of the input dictionary.
# (UC): The user can select which data wants to use for the analysis.
# Check Cobaya's documentation to see the list of the current available data experiments
# In this DEMO, we load the Euclid-Likelihood as an external function, and name it 'Euclid'

info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read
                     # in the config folder
                     #
                     # Select which observables want to use during the analysis
                     # by setting them to True or False
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': False}, 'CG': {'CG': False},
                         'add_phot_RSD': False, 'matrix_transform_phot': False
                     },
                     # Plot the selected observables matrx
                     'plot_observables_selection': True,  
                     # Switch to allow for matrix transformations of theory and data vectors
                     'matrix_transform_phot' : False, # 'BNT' or 'BNT-test,
                      # Nonlinear flags
                      # With this, the user can specify which nonlinear model they want
                      # For the time-being the available options are: 
                      # NL_flag_phot_matter
                        # 0 -> linear-only
                        # 1 -> Takahashi
                        # 2 -> Mead2016 (includes baryon corrections)
                        # 3 -> Mead2020 (w/o baryon corrections)
                        # 4 -> Mead2020_feedback (includes baryon corrections)
                        # 5 -> EE2
                        # 6 -> Bacco (matter)
                     'NL_flag_phot_matter': nlflag,
                      # NL_flag_spectro
                        # 0 -> linear-only
                        # 1 -> EFT
                     'NL_flag_spectro': 0,
                      # Baryon flag
                      # With this, the user can specify which baryon model they want
                      # For the time-being the available options are: 
                            #0 -> No baryonic feedback
                            #1 -> Mead2016 (baryons)
                            #2 -> Mead2020_feedback
                            #3 -> BCemu baryons
                            #4 -> Bacco baryons
                     'NL_flag_phot_baryon': barflag,
                      # Intrinsic alignment flag
                         #     0 -> NLA
                         #     1 -> TATT
                     'IA_flag': 0,
                      # IR-resummation method
                         #     'DST' -> Discrete sine transform
                         #     'EH' -> Eisenstein-Hu
                     'IR_resum': 'DST',
                     # This flag sets the redshift evolution for baryonic parameters for emulators
                     # The options are:
                            # True -> use X(z) = X_0 * (1+z)^(-nu_X), no. of params: 7*2 = 14
                            # False -> use X_i at each redshift bin i and interpolate, no. of params: 7*10 = 70
                     'Baryon_redshift_model': False,
                     'solver': 'camb',
                     'params': {
                                # (UC): galaxy bias parameters:
                                # The bias parameters below are currently fixed to the
                                # values used by the Inter Science Taskforce: Forecast (IST:F)
                                # and presented in the corresponding IST:F paper (arXiv: 1910.09273).
                                # However, they can be changed by the user and even sample over them by putting a prior
                                # Photometric bias parameters
                                'b1_photo_bin1': 1.0997727037892875,
                                'b1_photo_bin2': 1.220245876862528,
                                'b1_photo_bin3': 1.2723993083933989,
                                'b1_photo_bin4': 1.316624471897739,
                                'b1_photo_bin5': 1.35812370570578,
                                'b1_photo_bin6': 1.3998214171814918,
                                'b1_photo_bin7': 1.4446452851824907,
                                'b1_photo_bin8': 1.4964959071110084,
                                'b1_photo_bin9': 1.5652475842498528,
                                'b1_photo_bin10': 1.7429859437184225,
                                # 'b1_photo_bin1': 1.03047,
                                # 'b1_photo_bin2': 1.06699,
                                # 'b1_photo_bin3': 1.17363,
                                # 'b1_photo_bin4': 1.23340,
                                # 'b1_photo_bin5': 1.27510,
                                # 'b1_photo_bin6': 1.28574,
                                # 'b1_photo_bin7': 1.39434,
                                # 'b1_photo_bin8': 1.49170,
                                # 'b1_photo_bin9': 1.52334,
                                # 'b1_photo_bin10': 1.54639,                         
                                # Magnification bias parameters
                                'magnification_bias_1': 0.0,
                                'magnification_bias_2': 0.0,
                                'magnification_bias_3': 0.0,
                                'magnification_bias_4': 0.0,
                                'magnification_bias_5': 0.0,
                                'magnification_bias_6': 0.0,
                                'magnification_bias_7': 0.0,
                                'magnification_bias_8': 0.0,
                                'magnification_bias_9': 0.0,
                                'magnification_bias_10': 0.0,
                                # Shear calibration multiplicative bias parameters                                 
                                'multiplicative_bias_1': 0.0,
                                'multiplicative_bias_2': 0.0,
                                'multiplicative_bias_3': 0.0,
                                'multiplicative_bias_4': 0.0,
                                'multiplicative_bias_5': 0.0,
                                'multiplicative_bias_6': 0.0,
                                'multiplicative_bias_7': 0.0,
                                'multiplicative_bias_8': 0.0,
                                'multiplicative_bias_9': 0.0,
                                'multiplicative_bias_10': 0.0,
                                # Spectroscopic bias parameters
                                'b1_spectro_bin1': 1.46,
                                'b1_spectro_bin2': 1.61,
                                'b1_spectro_bin3': 1.75,
                                'b1_spectro_bin4': 1.90,
                                # Intrinsic alignment parameters
                                'a1_ia': 1.72,
                                'a2_ia': 0,
                                'b1_ia': 0,
                                'eta1_ia': -0.41,
                                'eta2_ia': 0,
                                'beta1_ia': 0,
                                # 'pivot_redshift': 0.,
                                # Redshift distributions nuisance parameters: shifts
                                'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
                                'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
                                'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
                                'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
                                'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
                                'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
                                'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
                                'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
                                'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
                                'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
                                'gamma_MG': 0.55,
                                'sigma_z': 0.002}, 
                     # k values for extrapolation of the matter power spectrum and size k-array
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 1E-5,   
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Add RSD to photometric probes
                     'add_phot_RSD': False,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     # Use redshift-dependent purity for GCspectro or not
                     'f_out_z_dep': False,
                     # Add spectroscopic redshift errors
                     'GCsp_z_err' : True,
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'scale_cuts_fourier': 'GCspectro-Fourier.yaml',
                            'Fourier': True},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss',
                            'photo_data': 'standard',
                            'Fourier': True},  # or 'BNT-Gauss' if BNT selected above
                        'cmbx': {
                              'root_CMBlens': 'Cls_kCMB.dat',
                              'root_CMBlensxWL': 'Cls_kCMBxWL.dat',
                              'root_CMBlensxGC': 'Cls_kCMBxGC.dat',
                              'root_CMBisw': 'Cls_{:s}_ISWxGC.dat',
                              'ISW_model': 'zNLA',
                              'cov_7x2pt': 'Cov_7x2pt_WL_GC_CMBX.npy',}}, 
                            }} 

In [ ]:
t1 = time.time()

# Second: create an instance of the `model` wrapper called model
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)

In [ ]:
# Let's run Cobaya
# the function run returns
# info_updated: an information dictionary updated with the defaults, 
# equivalent to the updated yaml file produced by the shell invocation
# samples: a sampler object, with a sampler.products() 
# being a dictionary of results. 
# For the mcmc sampler, the dictionary contains only one chain under the key sample.

t1 = time.time()
info_updated, samples = run(info)
print(time.time()-t1)

# Including CMB Probes 

In [ ]:
info_cmbx = copy.deepcopy(info)
info_cmbx['likelihood']['Euclid']['observables_selection']= {
                    'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                     'GCphot': {'GCphot': True, 'GCspectro': False},
                    'GCspectro': {'GCspectro': False},
                    'CMBlens': {'CMBlens':True, 'WL':True, 'GCphot':True},
                    'CMBisw': {'GCphot':True}, 'CG': {'CG': False}, 'add_phot_RSD': False, 'matrix_transform_phot': False
                     }

In [ ]:
model_cmbx = get_model(info_cmbx)

In [ ]:
t1 = time.time()
info_updated, samples = run(info_cmbx)
print(time.time() - t1)

# Sampling the posterior with As parameter

In [ ]:
# As = np.linspace(2.125e-9, 2.127e-9, num=5)
# #As = np.linspace(2.105e-9, 2.115e-9, num=5)
# log = np.zeros_like(As)
# log_cmbx = np.zeros_like(As)
# for i, A in enumerate(As):
#     log[i] = model.logposterior([A]).loglikes[0]
#     log_cmbx[i] = model_cmbx.logposterior([A]).loglikes[0]
#     print(i, m, log[i], log_cmbx[i])

In [ ]:
# plt.plot(As, log, label="3x2pt")
# plt.plot(As, log_cmbx, label="7x2pt")
# plt.axvline(x=2.12605e-9, c='r', label="fiducial")
# plt.legend()
# plt.xlabel("$A_s$")
# plt.ylabel("log posterior")

# Sampling the posterior with neutrino mass parameter

In [ ]:
mnu = np.linspace(0.055, 0.065, num=10)
log = np.zeros_like(mnu)
log_cmbx = np.zeros_like(mnu)
for i, m in enumerate(mnu):
    log[i] = model.logposterior([m]).loglikes[0]
    log_cmbx[i] = model_cmbx.logposterior([m]).loglikes[0]
    print(i, m, log[i], log_cmbx[i])

In [ ]:
plt.plot(mnu, np.exp(log), label="3x2pt")
plt.plot(mnu, np.exp(log_cmbx), label="7x2pt")
plt.axvline(x=0.06, c='r', label="fiducial")
plt.legend()
plt.xlabel(r"$\sum m_\nu$")
plt.ylabel("log posterior")

# Testing the Cls computations 

In [ ]:
info['likelihood']['Euclid']['params']['a1_ia'] = 0
info['likelihood']['Euclid']['params']['eta1_ia'] = 0
model = get_model(info)
info_cmbx['likelihood']['Euclid']['params']['a1_ia'] = 0
info_cmbx['likelihood']['Euclid']['params']['eta1_ia'] = 0
model_cmbx = get_model(info_cmbx)

In [ ]:
logposterior_cmbx = model_cmbx.logposterior([2.12605e-9])
like_cmbx = model_cmbx.likelihood['Euclid']

In [ ]:
photo = Photo(like_cmbx.cosmo.cosmo_dic, like_cmbx.likefinal.data_ins.nz_dict_WL, like_cmbx.likefinal.data_ins.nz_dict_GC_Phot, None)

In [ ]:
ells = np.unique(np.logspace(1, 3, num=100).astype('int'))

In [ ]:
def pp2kk(ells):
    return (ells*(ells+1))**2 /4

In [ ]:
cmbx = CMBX(photo)
cmbx.cmbx_update(photo)

cmbxdict = like_cmbx.likefinal.create_photoxcmb_data()
ckk_dat = like_cmbx.likefinal.data_ins.data_dict['kCMB']

In [ ]:
cl_kk = cmbx.Cl_kCMB(ells)
plt.loglog(ells, cl_kk, label='CLOE')

plt.loglog(ells, pp2kk(ells)*like_cmbx.cosmo.cosmo_dic['Cl']['pp'][ells], label='CAMB linear', ls='--')
plt.loglog(ckk_dat['ells'], cmbxdict['kCMB'], label='Stored data dict')
plt.legend()
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa \kappa}$')

In [ ]:
plt.plot(ells, cl_kk/(pp2kk(ells)*like_cmbx.cosmo.cosmo_dic['Cl']['pp'][ells]))

plt.xlabel('$L$')
plt.ylabel(r'$C_L^{\kappa \kappa, \mathrm{CLOE}} / C_L^{\kappa \kappa, \rm CAMB}$, linear')

In [ ]:
ntomowl = photo.nz_WL.get_num_tomographic_bins()
ntomogc = photo.nz_GC.get_num_tomographic_bins()

In [ ]:
cl_kgi = np.zeros([ntomogc, len(ells)])
cl_kwi = np.zeros([ntomowl, len(ells)])

for ibin in range(ntomogc):
    cl_kgi[ibin] = cmbx.Cl_kCMB_X_GC_phot(ells, ibin+1)
        
        
for ibin in range(ntomowl):
    cl_kwi[ibin] = cmbx.Cl_kCMB_X_WL(ells, ibin+1)

In [ ]:
plt.figure()

for ibin in range(ntomogc):
    plt.loglog(ells, cl_kgi[ibin], label='Bin {}'.format(ibin+1))
plt.legend(ncol=2)

plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{GC photo}}$')

In [ ]:
plt.figure()

for ibin in range(ntomowl):
    plt.loglog(ells, cl_kwi[ibin], label='Bin {}'.format(ibin+1))
plt.legend(ncol=2)

plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{WL}}$')

In [ ]:
cl_tgi = np.zeros([ntomogc, len(ells)])

for ibin in range(ntomogc):
    cl_tgi[ibin] = cmbx.Cl_ISWxGC(ells, ibin+1)

In [ ]:
def fact_ell(ell):
    return ell*(ell+1)/(2.*np.pi)

In [ ]:
plt.figure()

for ibin in range(ntomogc):
    plt.plot(ells, fact_ell(ells)*cl_tgi[ibin], label='Bin {}'.format(ibin+1))
plt.legend(ncol=2)
plt.xlim(2, 200)

plt.xlabel('$\ell$')
plt.ylabel('$C_\ell^{T, \mathrm{GC photo}}$')


# Additional comparisons with CAMB

In [ ]:
import camb
from camb import model as model_nl
from camb.sources import GaussianSourceWindow, SplinedSourceWindow

In [ ]:
# Cosmology 
fiducial_ffp10_with_H0  = {
#     'logA':np.log(2.119631e-09 *1e10),
    'As': 2.119631e-09,
    'ns': 0.9636852,
#     'theta_MC_100':0.010407224216796775 *100,
    'H0': 67.01904,
    'ombh2': 0.02216571,
    'omch2': 0.1202944,
    'tau':0.06018107,
    'mnu':0.06
}

## Get the Cls from CLOE

In [ ]:
info = {
    #'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
'params': {
        # (UC): each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
        # with the key 'prior', 'latex'...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the parameter ns
        # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': fiducial_ffp10_with_H0['ombh2'], #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': fiducial_ffp10_with_H0['omch2'], #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': fiducial_ffp10_with_H0['H0'], #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': fiducial_ffp10_with_H0['tau'], #optical depth
        'mnu': fiducial_ffp10_with_H0['mnu'], #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': fiducial_ffp10_with_H0['As'],
        'ns': fiducial_ffp10_with_H0['ns'], # primordial power spectrum tilt 
        'w': -1.0, #Dark energy fluid model
        'wa': 0.0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{
                          'num_massive_neutrinos': 1,
                          'dark_energy_model': 'ppf',
                          'bbn_predictor': 'PArthENoPE_880.2_standard.dat',
                          'lens_potential_accuracy': 4,
                          'DoLensing': True,
                          'NonLinear': 'NonLinear_both',
                          'kmax': 10,
                          'min_l': 2,
                          'max_l': 5000,
                          'max_eta_k': 100000,
                          'num_nu_massless': 2.03066666667,
                          'num_nu_massive': 1,
                          'nu_mass_eigenstates': 1,
                          'share_delta_neff': False,
                          'nu_mass_degeneracies': [1.01533333333],
                          'nu_mass_fractions': [1.0],
                          'nu_mass_numbers': [1],
                          'halofit_version': 'takahashi',
                          'HMCode_A_baryon': 3.13,
                          'HMCode_eta_baryon': 0.603,
                          'HMCode_logT_AGN': 7.8,
                          'scalar_initial_condition': 'initial_adiabatic',
                          'MassiveNuMethod': 'Nu_trunc'
                }
               }
              },
    'sampler': {'evaluate': None},  
    'output': 'chains/my_euclid_experiment',
    'debug': False,
    'timing': True,
    'force': True,
    }

In [ ]:
nlflag = 0 #3 to match default CAMB Halofit config
barflag = 0 

info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': False}, 
                        'CMBlens': {'CMBlens':True, 'WL':True, 'GCphot':True},
                        'CMBisw': {'GCphot':True}, 'CG': {'CG': False},
                         'CG': {'CG': False},
                         'add_phot_RSD': False, 'matrix_transform_phot': False
                     },
                    # Switch to allow for matrix transformations of theory and data vectors
                    'matrix_transform_phot' : False, # 'BNT' or 'BNT-test,
                      # Nonlinear flags
                      # With this, the user can specify which nonlinear model they want
                      # For the time-being the available options are: 
                      # NL_flag_phot_matter
                        # 0 -> linear-only
                        # 1 -> Takahashi
                        # 2 -> Mead2016 (includes baryon corrections)
                        # 3 -> Mead2020 (w/o baryon corrections)
                        # 4 -> Mead2020_feedback (includes baryon corrections)
                        # 5 -> EE2
                        # 6 -> Bacco (matter)
                     'NL_flag_phot_matter': nlflag,
                      # NL_flag_spectro
                        # 0 -> linear-only
                        # 1 -> EFT
                     'NL_flag_spectro': 0,
                      # Baryon flag
                      # With this, the user can specify which baryon model they want
                      # For the time-being the available options are: 
                            #0 -> No baryonic feedback
                            #1 -> Mead2016 (baryons)
                            #2 -> Mead2020_feedback
                            #3 -> BCemu baryons
                            #4 -> Bacco baryons
                     'NL_flag_phot_baryon': barflag,
                      # Intrinsic alignment flag
                         #     0 -> NLA
                         #     1 -> TATT
                     'IA_flag': 0,
                      # IR-resummation method
                         #     'DST' -> Discrete sine transform
                         #     'EH' -> Eisenstein-Hu
                     'IR_resum': 'DST',
                     # This flag sets the redshift evolution for baryonic parameters for emulators
                     # The options are:
                            # True -> use X(z) = X_0 * (1+z)^(-nu_X), no. of params: 7*2 = 14
                            # False -> use X_i at each redshift bin i and interpolate, no. of params: 7*10 = 70
                     'Baryon_redshift_model': False,
                     'solver': 'camb',
                     'params': {
                                # (UC): galaxy bias parameters:
                                # The bias parameters below are currently fixed to the
                                # values used by the Inter Science Taskforce: Forecast (IST:F)
                                # and presented in the corresponding IST:F paper (arXiv: 1910.09273).
                                # However, they can be changed by the user and even sample over them by putting a prior
                                # Photometric bias parameters
                                'b1_photo_bin1': 1.0997727037892875,
                                'b1_photo_bin2': 1.220245876862528,
                                'b1_photo_bin3': 1.2723993083933989,
                                'b1_photo_bin4': 1.316624471897739,
                                'b1_photo_bin5': 1.35812370570578,
                                'b1_photo_bin6': 1.3998214171814918,
                                'b1_photo_bin7': 1.4446452851824907,
                                'b1_photo_bin8': 1.4964959071110084,
                                'b1_photo_bin9': 1.5652475842498528,
                                'b1_photo_bin10': 1.7429859437184225,
                                # Magnification bias parameters
                                'magnification_bias_1': 0.0,
                                'magnification_bias_2': 0.0,
                                'magnification_bias_3': 0.0,
                                'magnification_bias_4': 0.0,
                                'magnification_bias_5': 0.0,
                                'magnification_bias_6': 0.0,
                                'magnification_bias_7': 0.0,
                                'magnification_bias_8': 0.0,
                                'magnification_bias_9': 0.0,
                                'magnification_bias_10': 0.0,
                                # Shear calibration multiplicative bias parameters                                 
                                'multiplicative_bias_1': 0.0,
                                'multiplicative_bias_2': 0.0,
                                'multiplicative_bias_3': 0.0,
                                'multiplicative_bias_4': 0.0,
                                'multiplicative_bias_5': 0.0,
                                'multiplicative_bias_6': 0.0,
                                'multiplicative_bias_7': 0.0,
                                'multiplicative_bias_8': 0.0,
                                'multiplicative_bias_9': 0.0,
                                'multiplicative_bias_10': 0.0,
                                # Spectroscopic bias parameters
                                'b1_spectro_bin1': 1.46,
                                'b1_spectro_bin2': 1.61,
                                'b1_spectro_bin3': 1.75,
                                'b1_spectro_bin4': 1.90,
                                # Intrinsic alignment parameters
                                 'a1_ia': 0, #1.72,
                                 'a2_ia': 0, #2,
                                 'b1_ia': 0, #1,
                                 'eta1_ia': 0, #-0.41,
                                 'eta2_ia': 0, #1,
                                 'beta1_ia': 0.0,
                                 'pivot_redshift': 0.,
                                # Redshift distributions nuisance parameters: shifts
                                'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
                                'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
                                'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
                                'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
                                'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
                                'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
                                'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
                                'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
                                'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
                                'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
                                'gamma_MG': 0.55,
                                'sigma_z': 0.002}, 
                     # k values for extrapolation of the matter power spectrum and size k-array
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 1E-5,   
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Add RSD to photometric probes
                     'add_phot_RSD': False,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     # Use redshift-dependent purity for GCspectro or not
                     'f_out_z_dep': False,
                     # Add spectroscopic redshift errors
                     'GCsp_z_err' : True,
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'scale_cuts_fourier': 'GCspectro-Fourier.yaml',
                            'Fourier': True},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss',
                            'photo_data': 'standard',
                            'Fourier': True},  # or 'BNT-Gauss' if BNT selected above
                        'cmbx': {
                              'root_CMBlens': 'Cls_kCMB.dat',
                              'root_CMBlensxWL': 'Cls_kCMBxWL.dat',
                              'root_CMBlensxGC': 'Cls_kCMBxGC.dat',
                              'root_CMBisw': 'Cls_{:s}_ISWxGC.dat',
                              'ISW_model': 'zNLA',
                              'cov_7x2pt': 'Cov_7x2pt_WL_GC_CMBX.npy',}}, 
                            }} 

In [ ]:
model_euc = get_model(info)

In [ ]:
model_euc.logposterior([])

In [ ]:
like = model_euc.likelihood['Euclid']
photo = Photo(like.cosmo.cosmo_dic, like.likefinal.data_ins.nz_dict_WL, like.likefinal.data_ins.nz_dict_GC_Phot)

cmblens = CMBX(photo)
cmblens.cmbx_update(photo)

## Get the CMB lensing Cls from CAMB

In [ ]:
pars = camb.CAMBparams()

pars.set_cosmology(H0=fiducial_ffp10_with_H0['H0'], ombh2=fiducial_ffp10_with_H0['ombh2'], omch2=fiducial_ffp10_with_H0['omch2'], mnu=fiducial_ffp10_with_H0['mnu'], omk=0, tau=fiducial_ffp10_with_H0['tau'])
pars.InitPower.set_params(As=fiducial_ffp10_with_H0['As'], ns=fiducial_ffp10_with_H0['ns'], r=0)


# pars.Want_CMB = True
# pars.Want_CMB_lensing = False
# pars.SourceTerms.DoPotential = False
pars.SourceTerms.limber_windows = True

pars.SourceTerms.counts_density = True
pars.SourceTerms.counts_lensing = False # magnification bias
pars.SourceTerms.counts_velocity = False
pars.SourceTerms.counts_radial = False
pars.SourceTerms.counts_redshift = False # redshift distortions
pars.SourceTerms.counts_potential = False
pars.SourceTerms.counts_timedelay = False
pars.SourceTerms.counts_ISW = False

pars.NonLinear = model_nl.NonLinear_none


# pars.NonLinearModel.set_params(
#     halofit_version='takahashi',
#     HMCode_A_baryon=3.13,
#     HMCode_eta_baryon=0.603,
#     HMCode_logT_AGN=7.8,
# )

# pars.SourceWindows = Win.get_camb_distributions()

# pars.set_for_lmax(2500, lens_potential_accuracy=8);
# pars.set_for_lmax(2500, lens_potential_accuracy=4);

# camb_results = camb.get_results(pars)

# lmax = 3000

In [ ]:
ntomowl = photo.nz_WL.get_num_tomographic_bins()
ntomogc = photo.nz_GC.get_num_tomographic_bins()

In [ ]:
sources = []
nuisance_params = like.cosmo.cosmo_dic['nuisance_parameters']
zz = np.linspace(0, 5, num=1000)
for ibin in range(ntomogc):
    sources.append(
        SplinedSourceWindow(source_type='counts', bias=nuisance_params[f'b1_photo_bin{ibin+1}'], z=zz, W=like.likefinal.data_ins.nz_dict_GC_Phot[f'n{ibin+1}'](zz))
    )
    
for ibin in range(ntomowl):
    sources.append(
        SplinedSourceWindow(source_type='lensing', bias=nuisance_params[f'b1_photo_bin{ibin+1}'], z=zz, W=like.likefinal.data_ins.nz_dict_GC_Phot[f'n{ibin+1}'](zz))
    )

In [ ]:
pars.SourceWindows = sources
camb_results = camb.get_results(pars)
clscambsources = camb_results.get_source_cls_dict()

lmax = 3000
camb_cls_pp = camb_results.get_lens_potential_cls(lmax=lmax, raw_cl = True)[:, 0] 

### CLOE Spectra

In [ ]:
lmin = 10
lmax = 2000

ls = np.arange(lmin, lmax+1)

In [ ]:
cl_kk_eucl = np.zeros(lmax+1)

for ell in ls:
    cl_kk_eucl[ell] = cmblens.Cl_kCMB(ell)

In [ ]:
plt.loglog(ls, cl_kk_eucl[ls], label='CLOE')
plt.loglog(ls, pp2kk(ls)*camb_cls_pp[ls], label='CAMB')

# plt.loglog(ells_euc, w(ells_euc)*like.cosmo.cosmo_dic['Cl']['pp'][ells_euc], label='CAMB linear (inaccurate for $L>1000$)')

plt.legend()
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa \kappa}$')

In [ ]:
plt.semilogx(ls, (camb_cls_pp[ls]*pp2kk(ls) / cl_kk_eucl[ls]  -1)*100, label='CAMB / CLOE')
plt.axhline(y=0, c='k')

plt.ylabel(r'$\Delta C_L^{\kappa, \kappa}$, \% diff', fontsize=15)
plt.legend()
plt.xlabel('$L$', fontsize=15)

## Comparing the CMB lensing Xcorr

In [ ]:
cl_kgi = np.zeros([ntomogc, lmax+1])
cl_kwi = np.zeros([ntomowl, lmax+1])

for ibin in range(ntomogc):
    for ell in ls:
        cl_kgi[ibin, ell] = cmblens.Cl_kCMB_X_GC_phot(ell, ibin+1)

for ibin in range(ntomowl):
    for ell in ls:
        cl_kwi[ibin, ell] = cmblens.Cl_kCMB_X_WL(ell, ibin+1)

In [ ]:
# Note from CAMB DEMO notebook 
# Note that P is CMB lensing, as a deflection angle power (i.e. PxP is [l(l+1)]^2C_l\phi\phi/2\pi)

In [ ]:
for ibin in range(ntomogc):
    plt.plot(ls, cl_kgi[ibin][ls], label='CLOE bin {}'.format(ibin+1))
    plt.plot(ls, clscambsources[(f'PxW{ibin+1}')][ls]/np.sqrt(ls*(ls+1))*np.pi, label='CAMB bin {}'.format(ibin+1), ls='--')
    
plt.loglog()
plt.legend(ncol=2)
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{GC photo}}$')
plt.title('Cross correlation of CMB lensing with galaxy density photo')

In [ ]:
for ibin in range(ntomogc):
    plt.plot(ls,  (clscambsources[(f'PxW{ibin+1}')][ls]/np.sqrt(ls*(ls+1))*np.pi / cl_kgi[ibin][ls] -1)*100, label='CAMB / CLOE bin {}'.format(ibin+1))

plt.semilogx()

plt.legend(ncol=2)
plt.xlabel('$L$')
plt.ylabel('$C_L^{\kappa, \mathrm{GC photo}}$ \% diff', fontsize=15)

plt.axhline(y=0, c='k')

plt.title('Relative difference of Kappa x GC photo')

In [ ]:
for ibin in range(ntomogc):
    plt.plot(ls, cl_kwi[ibin][ls], label='CLOE bin {}'.format(ibin+1))
    plt.plot(ls, clscambsources[(f'PxW{ibin+11}')][ls]/np.sqrt(ls*(ls+1))*np.pi, label='CAMB bin {}'.format(ibin+1), ls='--')
    
plt.loglog()

plt.legend(ncol=2)
plt.xlabel('$\ell$', fontsize=15)
plt.ylabel('$C_\ell^{\kappa, \mathrm{L}}$', fontsize=15)
plt.title('Cross correlation of CMB lensing with galaxy weak lensing')

In [ ]:
print('ell range:', ls)
print('WL prefactor:', (ls*(ls+1)/(ls+1/2)**2))
for ibin in range(ntomogc):
    plt.plot(ls,  (clscambsources[(f'PxW{ibin+11}')][ls]/np.sqrt(ls*(ls+1))*np.pi / cl_kwi[ibin][ls] -1)*100, label='CAMB / CLOE bin {}'.format(ibin+1))

plt.semilogx()
plt.legend(ncol=2)
plt.xlabel('$\ell$')
plt.ylabel('$C_\ell^{\kappa, \mathrm{L}}$ \% diff', fontsize=15)
plt.title('Relative difference of Kappa x Weak lensing')
plt.axhline(y=0, c='k')